$\newcommand{\is}{\mathrel{\mathop:=}}$
$\newcommand{\range}{\mathop{ran}}$
$\newcommand{\setof}[1]{\left \{ #1 \right \}}$
$\newcommand{\card}[1]{\left | #1 \right |}$
$\newcommand{\tuple}[1]{\left \langle #1 \right \rangle}$
$\newcommand{\emptytuple}{\left \langle \right \rangle}$
$\newcommand{\tuplecat}{\cdot}$
$\newcommand{\emptystring}{\varepsilon}$
$\newcommand{\String}[1]{\mathit{#1}}$
$\newcommand{\LeftEdgeSymbol}{\rtimes}$
$\newcommand{\RightEdgeSymbol}{\ltimes}$
$\newcommand{\LeftEdge}{\LeftEdgeSymbol}$
$\newcommand{\RightEdge}{\RightEdgeSymbol}$
$\newcommand{\mult}{\times}$
$\newcommand{\multisum}{\uplus}$
$\newcommand{\multimult}{\otimes}$
$\newcommand{\freqsymbol}{\mathrm{freq}}$
$\newcommand{\freq}[1]{\freqsymbol(#1)}$
$\newcommand{\prob}{P}$
$\newcommand{\count}[2]{\card{#2}_{#1}}$

# Simple Grammar Models with $n$-Grams

The bag-of-words model provides a method to extract from a text a very, very rough approximation of its meaning.
But the techniques it uses are much more general than that and can also be used to develop *grammars* - a system for separating well-formed from ill-formed utterances.
A grammar is an indispensable part of most language models.
Obviously a chatbot should not say gibberish, and a speech recognition model should not entertain *knzfr* as a possible English word you might have uttered a moment ago.
So some kind of grammar model is needed for most applications, but it does not necessarily have to be a very accurate model.
Just like the bag-of-words model is an impoverished model of meaning, the grammars we will see in this unit involve numerous simplifications compared to the implicit knowledge native speakers have about the rules of their language.
But it is nonetheless possible to get a surprising amount of work done with them.

## Bag-of-Sounds Models and Word-Final Devoicing

Every language has its own set of rules, and these rules affect all kinds of areas: the shape and pronunciation of words, how words can be built from other words, how sentences may be organized and how they may be interpreted, and much more.
As a concrete example, let us consider the *German devoicing rule*.

Sounds can be distinguished based on whether they are *voiced* or *voiceless*.
When a speaker utters a voiced sound, their vocal cords vibrate, giving the sounds a humming-like quality.
Examples of voiced sounds in English are *z* in *zeal* and *v* in *vault*.
A voiceless sound lacks the vibration of the vocal cords and thus sounds more like a hiss.
English examples include *s* in *seal* and *f* in *fault*.
Whereas English is fairly liberal in its use of voiced and voiceless sounds, German enforces an additional rule: no voiced sound may occur at the end of a word.

<div class=example>
    The German word for *read* is *lesen*.
    The *s* in *lesen* is voiced.
    Using English orthography, we could write it as *lezen*.
    The imperative form of *lesen* is *lies*.
    But in the imperative, the *s* is voiceless rather than voiced.
    This is because the *s* now occurs at the end of the word, and German does not allow any voiced sounds to occur in this position.
</div>

How could we go about specifying the German devoicing rule in a mathematically rigorous fashion?
Well, whatever formalization we choose, it should have the effect of forbidding all words that end in a voiced sound, and only those.
The $n$-gram bag-of-words model can actually take us a fair bit towards this goal.
Consider the word *lies*, pronounced [liːs] ("lees").
If we treat it as a string over a fixed set of sounds, we can use a bigram bag-of-words model to extract all sound bigrams: *li:* and *i:s*.

Before we move on, you might object that this is no longer a bag-of-words model but rather a bag-of-sounds model.
You are correct, but remember how we defined the function for the bag-of-words model:

$$
b_n: \Sigma^* \rightarrow \wp(\Sigma^n) \text{ with } t \mapsto \setof{ g \mid t = u \tuplecat g \tuplecat v, \text{ where } g \in \Sigma^n \text{ and } u,v \in \Sigma^*}.
$$

We said that $\Sigma$ is some fixed set of words, but nothing really hinges on that.
If we take $\Sigma$ to be a set of sounds instead, the function still behaves the same except that it now operates over smaller objects.
Mathematically, it does not really matter what kind of objects we take to be collected in $\Sigma$.
If $\Sigma$ is a set of words, $b_n$ is a model for extracting word $n$-grams from texts.
If $\Sigma$ is a set of sounds, $b_n$ is a model for extracting sound $n$-grams from words.
The difference is in what the function is used for, not in what the function does.

Let us return to [li:s] and its bigrams *li:* and *i:s*, then.
From these bigrams we can already tell that [li:s] does not violate the German devoicing rule, simply because it does not contain any voiced sounds to begin with.
But what about *lesen*, pronounced [le:zn]?
Its set of bigrams contains *le:*, *e:z*, and *zn*.
While there is a voiced *z* at the end of *e:z*, it is also apparent from *zn* that this *z* does not occur at the end of a word.
So again we can infer from the bigrams that the word does not violate the German ban against word-final voiced sounds.
Finally, consider the ill-formed [li:z].
Its set of bigrams consist of *li:* and *i:z*.
Since there is no bigram that starts with *z*, the *z* in *i:z* must be word-final and thus we have a violation of the German devoicing rule.

Our results so far suggest that $b_2$ in its current can recognize whether a word violates the German devoicing rule.
But now consider the German word for *buttocks*, which is *Gesäß* [geze:s].
Its ill-formed counterpart with word-final voicing would be [geze:z].
The bigrams for this word are *ge*, *ez*, *ze:*, and *e:z*.
While we can still infer that the word must contain at least two occurrences of *z*, it is no longer clear whether one of them is word-final.
To see this more clearly, ask yourself the following: does the set of bigrams change if we add another [e:] at the end of the word?
The answer is no - both [geze:z] and [geze:ze:] contain only the bigrams *ge*, *ez*, *ze:*, but the former violates the devoicing rule whereas the latter obeys it.
The bigrams of a string cannot tell us reliably whether a sound occurs in word-final position, and consequently $b_2$ cannot be used to reliably determine whether a word violates the devoicing rule.

## Augmented Strings and Strictly Local Grammars

The problem with $b_2$, or in fact, any $b_n$ is that decomposing a word into its $n$-grams destroys the information about which sounds occur at the beginning and the end of the word.
In some cases like the illicit [li:z] we can reconstruct this information from indirect evidence, but this isn't always possible.
However, as mathematically savvy language scientists we are free to come up with a formal device that will allow us to preserve this information.
And our solution can be very blunt: we can explictly mark the left and right edge of a word with the special symbols $\LeftEdge$ and $\RightEdge$ before we extract the bigrams with $b_2$.
So the input for $b_2$ isn't *geze:z*, it is *$\LeftEdge$geze:z$\RightEdge$*.
The bigrams of this word are
*$\LeftEdge$g*,
*ge*,
*ez*,
*ze:*,
*e:z*,
and
*z*$\RightEdge$.
Now it is perfectly clear that the word ends in a voiced sound and hence is ill-formed.

The idea is simple, and so is the formal definition.

<div class=definition>
    Let $s$ be a $\Sigma$-string.
    Then its *$n$-augmented* counterpart $\hat{s}^n$ is $\setof{\LeftEdge}^n \tuplecat s \tuplecat \setof{\RightEdge}^n$, where $\LeftEdge$ and $\RightEdge$ are distinguished symbols not contained in $\Sigma$.
</div>

**String**       | $\String{ab}$ | $\emptystring$
--:              | :--           | :--
**1-augmented**  | $\String{\LeftEdge ab \RightEdge}$                                       | $\String{\LeftEdge \RightEdge}$
**2-augmented**  | $\String{\LeftEdge\LeftEdge ab \RightEdge\RightEdge}$                    | $\String{\LeftEdge \LeftEdge \RightEdge \RightEdge}$
**3-augmented**  | $\String{\LeftEdge\LeftEdge\LeftEdge ab \RightEdge\RightEdge\RightEdge}$ | $\String{\LeftEdge \LeftEdge \LeftEdge \RightEdge \RightEdge \RightEdge}$

With augmented strings, we now keep track of certain positional information.
It only remains for us to give a formal mechanism to determine a word's well-formedness based on the output of $b_n$.
This is done by a *strictly local grammar*, which is just a set of forbidden $n$-grams.
The grammar rejects all strings whose augmented counterpart contains at least one of those forbidden $n$-grams.

<div class=definition>
    A *strictly $n$-local* (SL-$n$) grammar $G$ over $\Sigma$-strings is a subset of $(\Sigma \cup \setof{\LeftEdge, \RightEdge})^n$ (where $\Sigma \cap \setof{\LeftEdge, \RightEdge} = \emptyset$).
    A $\Sigma$-string $s$ is *well-formed with respect to $G$* iff $b_n(\hat{s}^{n-1}) \cap G = \emptyset$.
    We also say that $G$ *licenses* or *generates* $s$.
</div>

<div class=example>
    Suppose that the only voiced sounds in German are *z* and *v*.
    Then the word-final devoicing rule is captured by the strictly $2$-local grammar $\setof{\String{z\RightEdge}, \String{v\RightEdge}}$.
</div>

In [1]:
# b_n
def bag(string, n):
    return set(string[i:i+n] for i in range(len(string) - (n-1)))

# augmented string
def augment(string, n):
    return "_"*n + string + "_"*n

# well-formedness test
def is_wellformed(string, grammar, n):
    return False if bag(augment(string, n-1), n).intersection(grammar) else True

# pretty printer
def print_status(string, grammar, n):
    print(string, "is wellformed?", is_wellformed(string, grammar, n))

# the grammar for German word-final devoicing
grammar = set(["z_", "v_"])

print_status("le:sn", grammar, 2)
print_status("li:s", grammar, 2)
print_status("li:z", grammar, 2)
print_status("geze:s", grammar, 2)
print_status("geze:z", grammar, 2)
print_status("geze:ze", grammar, 2)

le:sn is wellformed? True
li:s is wellformed? True
li:z is wellformed? False
geze:s is wellformed? True
geze:z is wellformed? False
geze:ze is wellformed? True


If we wanted to express everything in terms of functions as we did for the bag-of-words model, our definitions would change only slightly.
First, we posit an augmentation function $a: \Sigma^* \times \mathbb{N} \rightarrow (\Sigma \cup \setof{\LeftEdge, \RightEdge})^*$ such that $\tuple{s,n} \mapsto \setof{\LeftEdge}^n \tuplecat s \tuplecat \setof{\RightEdge}^n$.
Second, there is an evaluation function $e: \wp((\Sigma \cup \setof{\LeftEdge, \RightEdge})^n) \times \wp((\Sigma \cup \setof{\LeftEdge, \RightEdge})^n) \rightarrow \setof{\top, \bot}$ such that $\tuple{B,G} \mapsto \top$ iff $B \cap G = \emptyset$.
Determining whether a strictly $n$-local grammar $G$ licenses string $s$ then corresponds to computing $e(b_n(a(s, n-1)), G)$.
If you are familiar with the programming language Python, you will see that this is also how the code above is designed.
Also note that this complex function has as its co-domain $\setof{\top, \bot}$, so it defines a set - the set of all strings that are well-formed with respect to a specific grammar.


## Isn't That a Tad Too Complicated?

At this point you are probably wondering why we are jumping through all these hoops.
Even if the definitions build on the bag-of-words model, they are quite a bit more complicated than anything we have seen so far.
The final paragraph of the previous section is downright bewildering with its abundance of symbols, subscripts, superscripts and so on.
And all of that just to forbid voiced sounds at the end?
We could have had that a lot more easily: the *German devoicing checker* function $d: \Sigma^* \rightarrow \setof{\top, \bot}$ maps every string $s$ to $\top$ unless $s \in \Sigma^* \times \setof{\text{v}, \text{z}}$.
There, just one function, and one that is easily defined within a single line.
No augmented strings, no grammars, just a clean formalization of the German ban against voiced sounds at the end of a word.
So, what gives?

It is true that German devoicing can be described very easily with a single function.
But this isn't the point of what we are doing here.

First, we wanted to see how the mathematical machinery of the bag-of-words model can be extended to a very different domain.
Intuitively, it is rather surprising that a model for the meaning of texts can also be used to regulate the shape of words.
That this is possible highlights a major advantage of mathematics: we can abstract away from the actual content or application area of the models without losing the rigor and specificity that is needed to understand how the model works.
This ability to relate the seemingly unrelated is one of the strongest arguments in favor of mathematical approaches.

Second, using similar models for different problems, rather than redesigning everything from scratch for each separate problem, means that it is easier to share insights, tools and so on.
This is particularly important as the models grow in complexity.
Studying the properties of a complex system can take a long time, so you should strive to import as many insights from models that are already well-understood.
Adding some minor modifications to an existing model is a reliable strategy along those lines.

Third, SL grammars are a much more flexible tool than the function $d$ above.
A huge number of sound-related phenomena can be handled with these grammars, for example intervocalic voicing and syllable templates.

<div class=example>
The intervocalic voicing rule forbids voiceless sounds to occur between two vowels.
This is captured by the SL-$3$ grammar $\setof{ \text{VsV} \mid \text{V is a vowel and s a voiceless sound}}$.

Syllable templates determine how consonants and vowels may be arranged in a single syllable.
For example, many languages use a CV template, according to which every syllable must contain exactly one consonant followed by exactly one syllable.
This can be expressed by an SL-$2$ grammar: $\setof{ \LeftEdge\text{V}, \text{CC}, \text{VV}, \text{C}\RightEdge}$.
</div>

The empirical breadth of SL grammars is another instance of sharing tools and insights: the programming implementation of SL grammars above, for example, will be able to handle all phenomena that can be captured by SL grammars,
An implementation of function $d$, on the other hand, won't help one bit when it comes to dealing with syllable templates.

The bottom line is that there is a trade-off between specificity and generality.
Models that only handle a few specific cases are often easy to define and figure out on an intuitive level, whereas more general models are more abstract and thus may seem a bit more impenetrable at first.
But the abstractness and generality actually makes many things easier and faster in the long run.
It's a bit like math itself: it comes with an above average buy-in cost, but it really pays off in the long run.
Remember: math is for lazy people, and abstract math is for really lazy people.


# Strictly Local Grammars with Deletion

Remember that we augmented our bag-of-worlds model with a $\mathrm{del}_S$ whose job it was to remove all members of $S$ from a text before it gets fed into $b_n$.
The purpose of this function was to deal with stop words, which make up a majority of every text but contribute little to no meaning.
Now that we have seen how the ideas behind the bag-of-words model can be generalized to SL grammars, it is worth asking whether we can also find a new usage for $\mathrm{del}_S$.

The original definition of $\mathrm{del}_S$ states that both its domain and its co-domain are $\Sigma^*$, and that $S$ is a subset of $\Sigma$.
In the examples above the SL grammars regulated the distribution of sounds of words, so $\Sigma$ is a fixed set of sounds.
Consequently, $S$ is a set of sounds that are to be removed.
While removing sounds form words may sound pretty useless for natural language, it actually is not.

Some languages put *long-distance* restrictions on the distribution of sounds.
For example, in Samala and Navajo word may contain either *s* or *ʃ*, but not both (the ʃ symbol represents the *sh* sound in words like *shoe* or *wish*).
It does no matter how far apart the two sounds are, it is simply not allowed for both of them to occur in the same word.
However, it is fine to have multiple instances of the same sound.

<div class=example>
    In Samala, *haʃxintilawaʃ* is a well-formed word, but *hasxintilawaʃ* is not because of the mismatch between *s* and *ʃ*.
</div>

If we wanted to capture this rule with an SL grammar, we would need at least $11$-ngrams, possibly more than that if there are even longer words in Samala.
An SL-$11$ grammar will be huge!
Think about it, the grammar must at least contain all $11$-grams of the form $\text{s} u_1 u_2 u_3 u_4 u_5 u_6 u_7 u_8 u_9 \text{ʃ}$.
Even if each intervening $u_i$ could only be instantiated as one of $3$ sounds (which is an preposterously low estimate) there would be $3^9 = 19,683$ instantiations of this template.
But then of course we also need the mirror version $\text{ʃ} u_1 u_2 u_3 u_4 u_5 u_6 u_7 u_8 u_9 \text{s}$, as well as variants like $u_1 u_2 u_3 \text{s} u_4 u_5 u_6 u_7 u_8 u_9 \text{ʃ}$ or $u_1 \text{s} u_2 u_3 u_4 u_5 \text{ʃ} u_6 u_7 u_8 u_9$.
In sum, the simple ban against *s* and *ʃ* in the same word would require $9 \cdot 8 \cdot 3^9 = 1,417,176$ distinct $11$-grams (and that's still assuming the unrealistic bound of at most three different sound values for each $u_i$).
Needless to say, that's neither elegant nor practical.

But suppose that we could just delete all sounds that are neither *s* nor *ʃ*.
Then the ban is captured by the very succinct SL-$2$ grammar $G \is \setof{\text{sʃ}, \text{ʃs}}$.

<div class=example>
    Consider again the well-formed *haʃxintilawaʃ* and the ill-formed *hasxintilawaʃ*.
    With all the irrelevant sounds removed, they become *ʃʃ* and *sʃ*, respectively.
    In order to know whether these are licensed by $G$ we need to compute the bigrams of their 1-augmented counterparts.
    The augmented strings are $\LeftEdge \text{*ʃʃ*} \RightEdge$ and $\LeftEdge \text{*sʃ*} \RightEdge$.
    The bigrams of $\LeftEdge \text{*ʃʃ*} \RightEdge$ $\LeftEdge \text{ʃ}$, ʃʃ, and $\text{ʃ} \RightEdge$, none of which are members of the grammar $\setof{\text{sʃ}, \text{ʃs}}$.
    Hence *haʃxintilawaʃ* is well-formed.
    The bigrams of $\LeftEdge \text{*sʃ*} \RightEdge$, on the other hand, are $\LeftEdge \text{s}$, sʃ, and $\text{ʃ} \RightEdge$, and since one of them is contained by the grammar,  *hasxintilawaʃ* is ill-formed.
</div>

<div class=definition>
    A *tier-based strictly $n$-local* (TSL-$n$) grammar $T$ over $\Sigma$-strings is a pair $\tuple{G, S}$ where $G$ is an SL-$n$ grammar and $S \subseteq \Sigma$ a set of symbols that should not be deleted.
    A $\Sigma$-string $s$ is *well-formed with respect to $T$* iff $b_n(\mathrm{del}_{\Sigma - S}(\hat{s}^{n-1})) \cap G = \emptyset$.
</div>

The name TSL is inspired by the resemblance between $\mathrm{del}_S$ and the so-called tier projection in a linguistic theory called *autosegmental phonology*.

The usage of the tuple notation $\tuple{G, S}$ to define TSL grammars follows the standard convention that whenever some object consists of multiple interacting components, those are given as the components of a tuple.
You should not attach too much meaning to this convention.
That SL-grammars are sets and TSL-grammars are tuples is just an artefact of long-standing notational conventions.
We will never treat TSL-grammars as if they were just a tuple, e.g. by concatenating multiple TSL grammars into a longer tuple.